<a href="https://colab.research.google.com/github/peremartra/LLMOptCost/blob/main/pruning_diltilgpt2_o1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Install necessary libraries
!pip install -q transformers
!pip install -q torch

# If you plan to upload to Hugging Face Hub
!pip install -q huggingface_hub

In [2]:
# Step 2: Import libraries
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch import nn
import os

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [3]:
# Step 3: Download the model and tokenizer
model_name = 'distilgpt2'
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
# Step 4: Test the model with a simple prompt
prompt = "Paris is the capital of"
inputs = tokenizer.encode(prompt, return_tensors='pt').to(device)
outputs = model.generate(inputs, max_length=10, num_return_sequences=1)
generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Generated text: {generated}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generated text: Paris is the capital of the United States.



In [5]:
# Step 5: Print the size of the original model
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

original_param_count = count_parameters(model)
print(f"Original model parameters: {original_param_count}")

Original model parameters: 81912576


In [6]:

# Step 6: Prune the MLP layers based on weight magnitude (adjusted for Conv1D layers)
import torch

prune_percent = 0.2  # For example, prune 20% of neurons

from transformers.models.gpt2.modeling_gpt2 import Conv1D

# Initialize new_intermediate_size
new_intermediate_size = None

for idx, block in enumerate(model.transformer.h):
    mlp = block.mlp

    # Get the weights of the c_fc layer (input projection)
    # c_fc.weight: [hidden_size, intermediate_size]
    c_fc_weight = mlp.c_fc.weight.data

    # Compute importance scores (L1 norm over input dimension for each output neuron)
    importance_scores = torch.sum(torch.abs(c_fc_weight), dim=0)  # Shape: [intermediate_size]

    # Determine the number of neurons to prune
    original_intermediate_size = c_fc_weight.size(1)  # This is intermediate_size
    num_neurons_to_prune = int(prune_percent * original_intermediate_size)

    # Get indices of neurons to keep (those with highest importance)
    _, indices_to_keep = torch.topk(importance_scores, original_intermediate_size - num_neurons_to_prune)

    # Sort indices to maintain order
    indices_to_keep, _ = torch.sort(indices_to_keep)

    # Create new Conv1D layers with reduced size
    # For c_fc: Conv1D(new_intermediate_size, hidden_size)
    new_c_fc = Conv1D(len(indices_to_keep), mlp.c_fc.weight.size(0)).to(device)

    # For c_proj: Conv1D(hidden_size, new_intermediate_size)
    new_c_proj = Conv1D(mlp.c_proj.weight.size(1), len(indices_to_keep)).to(device)

    # Copy over the weights and biases for neurons we're keeping
    # c_fc.weight: [hidden_size, intermediate_size], select columns
    new_c_fc.weight.data = mlp.c_fc.weight.data[:, indices_to_keep]
    new_c_fc.bias.data = mlp.c_fc.bias.data[indices_to_keep]

    # c_proj.weight: [intermediate_size, hidden_size], select rows
    new_c_proj.weight.data = mlp.c_proj.weight.data[indices_to_keep, :]
    new_c_proj.bias.data = mlp.c_proj.bias.data

    # Replace the layers in the model
    mlp.c_fc = new_c_fc
    mlp.c_proj = new_c_proj

    # Set new_intermediate_size if not already set
    if new_intermediate_size is None:
        new_intermediate_size = len(indices_to_keep)

# After pruning, update the model configuration
model.config.n_inner = new_intermediate_size

In [7]:
#Step 7: Recalculate the number of parameters
pruned_param_count = count_parameters(model)
print(f"Pruned model parameters: {pruned_param_count}")
print(f"Reduction in parameters: {original_param_count - pruned_param_count}")

Pruned model parameters: 76250268
Reduction in parameters: 5662308


In [8]:
# Step 8: Test the pruned model
outputs = model.generate(inputs, max_length=10, num_return_sequences=1)
generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Generated text after pruning: {generated}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated text after pruning: Paris is the capital of the United States, and


In [9]:
# Step 9: Save the pruned model
output_dir = './pruned_distilgpt2'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"Pruned model saved to {output_dir}")

Pruned model saved to ./pruned_distilgpt2


In [10]:
# Push the model to your Hugging Face repository
name_model_to_push="pruned_distilgpt2"

model.push_to_hub(name_model_to_push,
                  private=True,
                  use_temp_dir=False)




README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/oopere/pruned_distilgpt2/commit/1ca19c3fc9319fe913e88d47e92af02303c318d4', commit_message='Upload model', commit_description='', oid='1ca19c3fc9319fe913e88d47e92af02303c318d4', pr_url=None, pr_revision=None, pr_num=None)

In [11]:
tokenizer.push_to_hub(name_model_to_push,
                      private=False,
                      use_temp_dir=False)

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/oopere/pruned_distilgpt2/commit/1ca19c3fc9319fe913e88d47e92af02303c318d4', commit_message='Upload tokenizer', commit_description='', oid='1ca19c3fc9319fe913e88d47e92af02303c318d4', pr_url=None, pr_revision=None, pr_num=None)

In [12]:
# Step 11: Download the model from Hugging Face
pruned_model_name = 'oopere/pruned_distilgpt2'
pruned_model = AutoModelForCausalLM.from_pretrained(pruned_model_name).to(device)
pruned_tokenizer = AutoTokenizer.from_pretrained(pruned_model_name)

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/305M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [13]:
print(model.config)

GPT2Config {
  "_name_or_path": "distilgpt2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": 2458,
  "n_layer": 6,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.44.2",
  "use_cache": true,
  "

In [14]:
# Step 12: Test the pruned_loaded model
outputs = pruned_model.generate(inputs, max_length=10, num_return_sequences=1)
generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Generated text after pruning: {generated}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated text after pruning: Paris is the capital of the United States, and
